In [1]:
import sys, os, time, getpass

sys.path.append("/home/tatiane/lib/")

import pessoal
from pessoal import *
print('AppID: ', sc.applicationId)

# Pedir ao spark para exibir só erros, escondendo avisos (WARN) e infos
# O correto é resolver a causa como, por exemplo: ordenados = Window.partitionBy().orderBy(dfs[i].columns[0])
spark.sparkContext.setLogLevel("ERROR")

Tempo inicial da execucao: 2025-07-03 15:18:18.877222
User: tatiane
Node: Inspiron-3583
AppID:  local-1751566665880


# Censo escolar 2023

In [2]:
# Leitura com encoding apropriado para o conteúdo da base
censo = spark.read.csv('/home/tatiane/Documentos/censo_escolar2/projeto_2/microdados_censo_escolar_2023/dados/microdados_ed_basica_2023.csv', header = True, multiLine = True, sep = ';', encoding="latin1")
write_path = '/home/tatiane/Documentos/censo_escolar2/projeto_2/analise2_original/'

In [3]:
pessoal.completudeSchema(censo)

Qtd. registros: 217625 | Quantidade de colunas:  408
root
 |-- NU_ANO_CENSO: string (nullable = true)
 |-- NO_REGIAO: string (nullable = true)
 |-- CO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CO_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- CO_MUNICIPIO: string (nullable = true)
 |-- NO_REGIAO_GEOG_INTERM: string (nullable = true)
 |-- CO_REGIAO_GEOG_INTERM: string (nullable = true)
 |-- NO_REGIAO_GEOG_IMED: string (nullable = true)
 |-- CO_REGIAO_GEOG_IMED: string (nullable = true)
 |-- NO_MESORREGIAO: string (nullable = true)
 |-- CO_MESORREGIAO: string (nullable = true)
 |-- NO_MICRORREGIAO: string (nullable = true)
 |-- CO_MICRORREGIAO: string (nullable = true)
 |-- CO_DISTRITO: string (nullable = true)
 |-- NO_ENTIDADE: string (nullable = true)
 |-- CO_ENTIDADE: string (nullable = true)
 |-- TP_DEPENDENCIA: string (nullable = true)
 |-- TP_CATEGORIA_ESCOLA_PRIVADA: string (nullab

#### Criação de um identificador unico

In [4]:
censo = createId(censo, 'ID_UNICO')
censo.select(F.min(F.col('ID_UNICO')), F.max(F.col('ID_UNICO'))).show()

+-------------+-------------+
|min(ID_UNICO)|max(ID_UNICO)|
+-------------+-------------+
|            1|       217625|
+-------------+-------------+



#### Escolha de variáveis

In [5]:
analise2 = censo.select('ID_UNICO', 'NO_REGIAO', 'SG_UF', 'NO_MUNICIPIO', 'NO_ENTIDADE', 'TP_DEPENDENCIA', 'TP_LOCALIZACAO', 'TP_LOCALIZACAO_DIFERENCIADA', 'TP_SITUACAO_FUNCIONAMENTO', 'IN_INTERNET', 'IN_INTERNET_ALUNOS', 'IN_ALIMENTACAO', 'IN_EDUCACAO_INDIGENA', 'TP_INDIGENA_LINGUA')
completude(analise2)

for col in analise2.columns:
    print(col)
    analise2.groupBy(col).count().orderBy(F.col(col).asc()).show()
printTimer()

Qtd. registros: 217625 | Quantidade de colunas:  14
ID_UNICO


+--------+-----+
|ID_UNICO|count|
+--------+-----+
|       1|    1|
|       2|    1|
|       3|    1|
|       4|    1|
|       5|    1|
|       6|    1|
|       7|    1|
|       8|    1|
|       9|    1|
|      10|    1|
|      11|    1|
|      12|    1|
|      13|    1|
|      14|    1|
|      15|    1|
|      16|    1|
|      17|    1|
|      18|    1|
|      19|    1|
|      20|    1|
+--------+-----+
only showing top 20 rows

NO_REGIAO


+------------+-----+
|   NO_REGIAO|count|
+------------+-----+
|Centro-Oeste|11459|
|    Nordeste|76348|
|       Norte|25498|
|     Sudeste|75658|
|         Sul|28662|
+------------+-----+

SG_UF


+-----+-----+
|SG_UF|count|
+-----+-----+
|   AC| 1658|
|   AL| 3473|
|   AM| 6148|
|   AP|  950|
|   BA|19611|
|   CE|10648|
|   DF| 1319|
|   ES| 4037|
|   GO| 5047|
|   MA|13635|
|   MG|24293|
|   MS| 1939|
|   MT| 3154|
|   PA|12092|
|   PB| 5479|
|   PE|10219|
|   PI| 5849|
|   PR| 9999|
|   RJ|13229|
|   RN| 4867|
+-----+-----+
only showing top 20 rows

NO_MUNICIPIO


+-------------------+-----+
|       NO_MUNICIPIO|count|
+-------------------+-----+
|    Abadia de Goiás|    9|
|Abadia dos Dourados|    8|
|          Abadiânia|   13|
|         Abaetetuba|  212|
|             Abaeté|   24|
|            Abaiara|   18|
|              Abaré|   45|
|             Abatiá|    6|
|             Abaíra|    8|
|      Abdon Batista|    7|
|    Abel Figueiredo|   13|
|       Abelardo Luz|   23|
|         Abre Campo|   19|
|       Abreu e Lima|   95|
|        Abreulândia|    6|
|            Acaiaca|    8|
|          Acajutiba|   32|
|            Acarape|   17|
|             Acaraú|   60|
|              Acari|   20|
+-------------------+-----+
only showing top 20 rows

NO_ENTIDADE


...

+-----------+------+
|IN_INTERNET| count|
+-----------+------+
|       null| 37395|
|          0| 16375|
|          1|163855|
+-----------+------+

IN_INTERNET_ALUNOS


+------------------+------+
|IN_INTERNET_ALUNOS| count|
+------------------+------+
|              null| 37395|
|                 0|107148|
|                 1| 73082|
+------------------+------+

IN_ALIMENTACAO


+--------------+------+
|IN_ALIMENTACAO| count|
+--------------+------+
|          null| 37395|
|             0| 28828|
|             1|151402|
+--------------+------+

IN_EDUCACAO_INDIGENA


+--------------------+------+
|IN_EDUCACAO_INDIGENA| count|
+--------------------+------+
|                null| 37395|
|                   0|176602|
|                   1|  3628|
+--------------------+------+

TP_INDIGENA_LINGUA


+------------------+------+
|TP_INDIGENA_LINGUA| count|
+------------------+------+
|              null|213997|
|                 1|   151|
|                 2|   938|
|                 3|  2539|
+------------------+------+

Tempo de execucao: 0:00:47.059648


#### Tratamento das variáveis

- Remoção de numerações no nome da escola
- Transformando numerações em categorias

In [6]:
print('Quantidade de registros que iniciam com numerações que não fazem parte do nome: ', analise2.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).count())

analise2.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).select("NO_ENTIDADE").show(truncate=False)

Quantidade de registros que iniciam com numerações que não fazem parte do nome:  1771


+--------------------------------------------------------+
|NO_ENTIDADE                                             |
+--------------------------------------------------------+
|29092191 - COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL |
|0101501 CIEP HENFIL                                     |
|0101803 EDI ANTONIO RAPOSO TAVARES                      |
|0101004 EM BENJAMIM CONSTANT                            |
|0101003 EM DARCY VARGAS                                 |
|0101006 EM FRANCISCO BENJAMIN GALLOTTI                  |
|0101005 EM GENERAL MITRE                                |
|0101008 EM MARECHAL ESPERIDIAO ROSAS                    |
|0101007 EM MARECHAL MASCARENHAS DE MORAES               |
|0101012 EM PROFESSORA LAURA SYLVIA MENDES PEREIRA       |
|0101009 EM PROFESSOR WALTER CARLOS DE MAGALHAES FRAENKEL|
|0101001 EM VICENTE LICINIO CARDOSO                      |
|0102503 CIEP AVENIDA DOS DESFILES II                    |
|0102502 CIEP AVENIDA DOS DESFILES I                    

In [7]:
# substituição das numerações por vázio
analise2 = analise2.withColumn("NOME_ENTIDADE", F.regexp_replace(F.col("NO_ENTIDADE"), "^(\d+\.?){3,} (- )?", ''))

print('Validando as modificações: ')
analise2.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).select("NO_ENTIDADE", "NOME_ENTIDADE").show(truncate=False)

#removendo a variável anterior
analise2 = analise2.drop('NO_ENTIDADE')

Validando as modificações: 


+--------------------------------------------------------+------------------------------------------------+
|NO_ENTIDADE                                             |NOME_ENTIDADE                                   |
+--------------------------------------------------------+------------------------------------------------+
|29092191 - COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL |COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL    |
|0101501 CIEP HENFIL                                     |CIEP HENFIL                                     |
|0101803 EDI ANTONIO RAPOSO TAVARES                      |EDI ANTONIO RAPOSO TAVARES                      |
|0101004 EM BENJAMIM CONSTANT                            |EM BENJAMIM CONSTANT                            |
|0101003 EM DARCY VARGAS                                 |EM DARCY VARGAS                                 |
|0101006 EM FRANCISCO BENJAMIN GALLOTTI                  |EM FRANCISCO BENJAMIN GALLOTTI                  |
|0101005 EM GENERAL MITRE   

#### Criação e aplicação de dicionário

In [8]:
from pyspark.sql import functions as F
from itertools import chain

mapas = {
    "TP_DEPENDENCIA" : {1: "Federal", 2: "Estadual", 3: "Municipal", 4: "Privada"},
    "TP_LOCALIZACAO" : {1: "Urbana", 2: "Rural"},
    "TP_LOCALIZACAO_DIFERENCIADA" : {0: "Não se aplica", 1: "Área de assentamento", 2: "Terra índigena", 3: "comunidade remanescente de quilombos", 4: "comunidades tradicionais"},
    "TP_SITUACAO_FUNCIONAMENTO" : {1: "Em atividade", 2: "Paralisada", 3: "Extinta (ano do censo)", 4: "Extinta (anos anteriores)"},
    "IN_INTERNET" : {0: "Não", 1: "Sim"},
    "IN_INTERNET_ALUNOS" : {0: "Não", 1: "Sim"},
    "IN_ALIMENTACAO" : {0: "Não oferece", 1: "Oferece"},
    "IN_EDUCACAO_INDIGENA" : {0: "Não", 1: "Sim"},
    "TP_INDIGENA_LINGUA" : {1: "Somente em Língua Indígena", 2: "Somente em Língua Portuguesa", 3: "Língua Indígena e Portuguesa", 4: "Não aplicável"},
}

In [9]:
analise_final = analise2
for col, dic in mapas.items():
    mapping_expr = F.create_map([F.lit(x) for x in chain(*dic.items())])
    analise_final = analise_final.withColumn(col, mapping_expr[F.col(col)])

In [ ]:
for col in analise_final.columns:
    print(col)
    analise_final.groupBy(col).count().orderBy(F.col(col).asc()).show()

ID_UNICO


+--------+-----+
|ID_UNICO|count|
+--------+-----+
|       1|    1|
|       2|    1|
|       3|    1|
|       4|    1|
|       5|    1|
|       6|    1|
|       7|    1|
|       8|    1|
|       9|    1|
|      10|    1|
|      11|    1|
|      12|    1|
|      13|    1|
|      14|    1|
|      15|    1|
|      16|    1|
|      17|    1|
|      18|    1|
|      19|    1|
|      20|    1|
+--------+-----+
only showing top 20 rows

NO_REGIAO


+------------+-----+
|   NO_REGIAO|count|
+------------+-----+
|Centro-Oeste|11459|
|    Nordeste|76348|
|       Norte|25498|
|     Sudeste|75658|
|         Sul|28662|
+------------+-----+

SG_UF


+-----+-----+
|SG_UF|count|
+-----+-----+
|   AC| 1658|
|   AL| 3473|
|   AM| 6148|
|   AP|  950|
|   BA|19611|
|   CE|10648|
|   DF| 1319|
|   ES| 4037|
|   GO| 5047|
|   MA|13635|
|   MG|24293|
|   MS| 1939|
|   MT| 3154|
|   PA|12092|
|   PB| 5479|
|   PE|10219|
|   PI| 5849|
|   PR| 9999|
|   RJ|13229|
|   RN| 4867|
+-----+-----+
only showing top 20 rows

NO_MUNICIPIO


+-------------------+-----+
|       NO_MUNICIPIO|count|
+-------------------+-----+
|    Abadia de Goiás|    9|
|Abadia dos Dourados|    8|
|          Abadiânia|   13|
|         Abaetetuba|  212|
|             Abaeté|   24|
|            Abaiara|   18|
|              Abaré|   45|
|             Abatiá|    6|
|             Abaíra|    8|
|      Abdon Batista|    7|
|    Abel Figueiredo|   13|
|       Abelardo Luz|   23|
|         Abre Campo|   19|
|       Abreu e Lima|   95|
|        Abreulândia|    6|
|            Acaiaca|    8|
|          Acajutiba|   32|
|            Acarape|   17|
|             Acaraú|   60|
|              Acari|   20|
+-------------------+-----+
only showing top 20 rows

TP_DEPENDENCIA


...

+---------------------------+------+
|TP_LOCALIZACAO_DIFERENCIADA| count|
+---------------------------+------+
|                       null| 38161|
|              Não se aplica|168720|
|             Terra índigena|  3589|
|       comunidade remane...|  2602|
|       Área de assentamento|  4553|
+---------------------------+------+

TP_SITUACAO_FUNCIONAMENTO


+-------------------------+------+
|TP_SITUACAO_FUNCIONAMENTO| count|
+-------------------------+------+
|             Em atividade|180230|
|     Extinta (ano do c...|  5312|
|               Paralisada| 32083|
+-------------------------+------+

IN_INTERNET


+-----------+------+
|IN_INTERNET| count|
+-----------+------+
|       null| 37395|
|        Não| 16375|
|        Sim|163855|
+-----------+------+

IN_INTERNET_ALUNOS


+------------------+------+
|IN_INTERNET_ALUNOS| count|
+------------------+------+
|              null| 37395|
|               Não|107148|
|               Sim| 73082|
+------------------+------+

IN_ALIMENTACAO


+--------------+------+
|IN_ALIMENTACAO| count|
+--------------+------+
|          null| 37395|
|   Não oferece| 28828|
|       Oferece|151402|
+--------------+------+

IN_EDUCACAO_INDIGENA


In [ ]:
analise_final = analise_final.withColumnRenamed('NOME_ENTIDADE', 'NO_ENTIDADE')
pessoal.completudeSchema(analise_final)

#### Escrita

In [ ]:
writeSingleCsv(analise2, write_path)

### Finalização do notebook

In [ ]:
executionTime()

In [ ]:
sc.stop()